In [1]:
import os
import numpy as np
import pandas as pd
import re

os.chdir('.')

def listdir():
    for root, dirs, files in os.walk("."):
        for filename in files:
            print(filename)

In [2]:
data = pd.read_csv('SR_PartnerRatings_PS.csv', header = 0)
data

,TrialNumber,Partner,Trait,ran,order,Rating,sub
0,1,3,0,1,0,5,1001
1,2,2,0,1,1,-2,1001
2,3,1,0,1,2,5,1001
3,4,3,1,1,3,5,1001
4,5,2,1,1,4,0,1001
...,...,...,...,...,...,...,...
301,2,2,0,1,1,5,3218
302,3,1,0,1,2,5,3218
303,4,3,1,1,3,-5,3218
304,5,2,1,1,4,-5,3218


In [3]:
Trait = {0:'Win', 1:'Lose'}
Partner = {3:'Friend',2:'Stranger',1:'Comp'}

In [4]:
#This loop collapses the 6 rows of Shared Reward Partner Ratings into 1 row for each participant
collapsed = {}
for index, row in data.iterrows():
    if(not row['sub'] in collapsed):
        collapsed[row['sub']] = {}
    column_name = Trait[row['Trait']] + '-' + Partner[row['Partner']]
    collapsed[row['sub']][column_name] = row['Rating']

In [5]:
#PR = Partner Ratings from post-scan task
PR = pd.DataFrame(collapsed).T
PR = PR.reset_index().rename(columns = {'index':'sub'})
PR.to_csv('SR_PartnerRatings.csv',index=False)
pd.read_csv('SR_PartnerRatings.csv')

,sub,Win-Friend,Win-Stranger,Win-Comp,Lose-Friend,Lose-Stranger,Lose-Comp
0,1001,5,-2,5,5,0,4
1,1002,4,2,0,-3,-1,0
2,1006,3,2,5,-4,-1,-2
3,1007,4,4,3,0,0,0
4,1009,5,4,4,-5,-4,-4
5,1010,1,-2,-5,2,-3,-5
6,1011,5,5,2,-5,-5,-5
7,1012,3,0,0,-3,0,0
8,1013,5,-3,2,-4,1,-2
9,1015,5,3,0,-5,-3,0


In [7]:
#RC = REDCap data
RC = pd.read_csv('SR_REDCap_data.csv').rename(columns = {'participant_id':'sub', 'ios_p':'closeness_rating'})
#RS = Reward Sensitivity scores
RS = pd.read_csv('Reward_Sensitivity.csv').rename(columns = {'RealID':'sub'})
#ASRD = All Shared Reward Data
ASRD = RS.merge(PR, on = 'sub', how = 'right').sort_values('sub', ascending = True, ignore_index = True)
ASRD = ASRD.merge(RC, on = 'sub', how = 'left').sort_values('sub', ascending = True, ignore_index = True)
ASRD.drop('redcap_event_name', axis = 'columns').to_csv('AllSharedRewardData.csv', index = False)
pd.read_csv('AllSharedRewardData.csv')

,Unnamed: 0,sub,gender,SPSRWD,BISBAS_BAS,PC1,Win-Friend,Win-Stranger,Win-Comp,Lose-Friend,Lose-Stranger,Lose-Comp,screen_age,closeness_rating
0,0,1001,1,7.000000,44.000000,0.664463,5,-2,5,5,0,4,NaN,5.0
1,1,1002,2,15.000000,35.000000,-1.433882,4,2,0,-3,-1,0,NaN,5.0
2,6,1006,2,12.000000,37.000000,-1.725609,3,2,5,-4,-1,-2,NaN,5.0
3,7,1006,2,12.000000,37.000000,-1.725609,3,2,5,-4,-1,-2,NaN,5.0
4,8,1007,1,14.000000,37.000000,-0.485203,4,4,3,0,0,0,21.235207,5.0
5,10,1009,1,12.000000,40.000000,0.627714,5,4,4,-5,-4,-4,20.203016,6.0
6,11,1010,2,11.000000,37.000000,-2.345812,1,-2,-5,2,-3,-5,22.215377,6.0
7,12,1011,2,14.000000,39.000000,1.083679,5,5,2,-5,-5,-5,21.979917,5.0
8,13,1012,1,5.000000,33.000000,-9.204796,3,0,0,-3,0,0,22.064793,5.0
9,14,1013,1,16.000000,41.000000,3.892968,5,-3,2,-4,1,-2,18.579551,4.0
